# **Signal To Spike Conversion** - Analysis

This notebook conducts an analysis of the Inter-Spike Interval in the UP and DN spike trains to determine the optimal synaptic time constants

### Check WD (change if necessary) and file loading

In [1]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


### Declare the `INPUT_FOLDER` and `RESULTS_FOLDER`

In [2]:
# CAREFUL WITH THIS FOLDER TO NOT OVERWRITE THE FILES
DATASET_FILENAME = "seeg_filtered_subset_90-119_segment500_200"
INPUT_FOLDER = f"results/{DATASET_FILENAME}"
RESULTS_FOLDER = f"analysis/{DATASET_FILENAME}"

## Load the UP and DN spike trains in the Ripple and FR Band

In [3]:
import numpy as np
import math
from utils.io import preview_np_array
from utils.input import read_spike_events, MarkerType, band_to_file_name, BaselineAlgorithm

### Load the Baseline Thresholds from the output file from the baseline process

In [4]:
# Specify the chosen Baseline Algorithm
chosen_baseline_alg_suffix = BaselineAlgorithm.MEAN

In [5]:
# Load the Baseline Thresholds
BASELINE_FILE = f"baseline_results/{DATASET_FILENAME}_thresholds_{chosen_baseline_alg_suffix}.npy"
baseline_thresholds = np.load(BASELINE_FILE)

# preview_np_array(baseline_thresholds, "baseline_thresholds", edge_items=3)

baseline_ripple_thresh = round(baseline_thresholds[0], 4)
baseline_fr_thresh = round(baseline_thresholds[1], 4)

# For now, the UP and DN thresholds are the same (symmetric)
ripple_thresh_up = baseline_ripple_thresh
ripple_thresh_down = -baseline_ripple_thresh
fr_thresh_up = baseline_fr_thresh
fr_thresh_down = -baseline_fr_thresh

print("Ripple Thresholds: ", ripple_thresh_up, ripple_thresh_down)
print("FR Thresholds: ", fr_thresh_up, fr_thresh_down)

Ripple Thresholds:  4.7916 -4.7916
FR Thresholds:  2.3014 -2.3014


In [6]:
ripple_band_filename = band_to_file_name(MarkerType.RIPPLE)
fr_band_filename = band_to_file_name(MarkerType.FAST_RIPPLE)

# Call the function to read the spike events
ripple_up_spikes_file_path = f"{INPUT_FOLDER}/{ripple_band_filename}_up_spike_train_{ripple_thresh_up}.csv"
ripple_down_spikes_file_path = f"{INPUT_FOLDER}/{ripple_band_filename}_down_spike_train_{ripple_thresh_down}.csv"

ripple_up_spike_train = read_spike_events(ripple_up_spikes_file_path)
ripple_down_spike_train = read_spike_events(ripple_down_spikes_file_path)


preview_np_array(ripple_up_spike_train, "ripple_up_spike_train", edge_items=3)
preview_np_array(ripple_down_spike_train, "ripple_down_spike_train", edge_items=3)

ripple_up_spike_train Shape: (2545, 2).
Preview: [[ 9.44335938e+02 -1.00000000e+00]
 [ 9.51171875e+02 -1.00000000e+00]
 [ 9.52636719e+02 -1.00000000e+00]
 ...
 [ 1.19092285e+05 -1.00000000e+00]
 [ 1.19093750e+05 -1.00000000e+00]
 [ 1.19106445e+05 -1.00000000e+00]]
ripple_down_spike_train Shape: (2530, 2).
Preview: [[ 9.47265625e+02 -1.00000000e+00]
 [ 9.48730469e+02 -1.00000000e+00]
 [ 9.56054688e+02 -1.00000000e+00]
 ...
 [ 1.19087891e+05 -1.00000000e+00]
 [ 1.19089355e+05 -1.00000000e+00]
 [ 1.19100098e+05 -1.00000000e+00]]


In [7]:
fr_up_spikes_file_path = f"{INPUT_FOLDER}/{fr_band_filename}_up_spike_train_{fr_thresh_up}.csv"
fr_down_spikes_file_path = f"{INPUT_FOLDER}/{fr_band_filename}_down_spike_train_{fr_thresh_down}.csv"

fr_up_spike_train = read_spike_events(fr_up_spikes_file_path)
fr_down_spike_train = read_spike_events(fr_down_spikes_file_path)

preview_np_array(fr_up_spike_train, "fr_up_spike_train", edge_items=3)
preview_np_array(fr_down_spike_train, "fr_down_spike_train", edge_items=3)

fr_up_spike_train Shape: (3971, 2).
Preview: [[ 1.00195312e+03 -1.00000000e+00]
 [ 1.00244141e+03 -1.00000000e+00]
 [ 1.00537109e+03 -1.00000000e+00]
 ...
 [ 1.19012207e+05 -1.00000000e+00]
 [ 1.19015137e+05 -1.00000000e+00]
 [ 1.19015625e+05 -1.00000000e+00]]
fr_down_spike_train Shape: (4045, 2).
Preview: [[ 1.00097656e+03 -1.00000000e+00]
 [ 1.00341797e+03 -1.00000000e+00]
 [ 1.00390625e+03 -1.00000000e+00]
 ...
 [ 1.19013672e+05 -1.00000000e+00]
 [ 1.19014160e+05 -1.00000000e+00]
 [ 1.19016602e+05 -1.00000000e+00]]


### Load the Annotated Data
The annotated data is needed to separate the intervals of HFO from the normal activity to calculate the Inter-Spike Interval (ISI)

In [8]:
# load the npy object
ripple_ground_truth = np.load(f"{INPUT_FOLDER}/{ripple_band_filename}_ground_truth.npy")
fr_ground_truth = np.load(f"{INPUT_FOLDER}/{fr_band_filename}_ground_truth.npy")

preview_np_array(ripple_ground_truth, "ripple_ground_truth", edge_items=3)
preview_np_array(fr_ground_truth, "fr_ground_truth", edge_items=3)

ripple_ground_truth Shape: (222,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Spike+Ripple', 116216.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Ripple', 119000.  , 0.)]
fr_ground_truth Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Fast-Ripple', 119000.  , 0.)]


## Iterate the UP and DN Spike Trains to Calculate the Inter-Spike Interval (ISI) in the HFO and Normal Activity

Define the lists to store:
- `Ripple_HFO_UP` - ISI of the UP train in the Ripple band during an HFO event 
- `Ripple_HFO_DN` - ISI of the DN train in the Ripple band during an HFO event
- `Ripple_Noise_UP` - ISI of the UP train in the Ripple band during normal activity
- `Ripple_Noise_DN` - ISI of the DN train in the Ripple band during normal activity

- `FR_HFO_UP` - ISI of the UP train in the Fast Ripple band during an HFO event
- `FR_HFO_DN` - ISI of the DN train in the Fast Ripple band during an HFO event
- `FR_Noise_UP` - ISI of the UP train in the Fast Ripple band during normal activity
- `FR_Noise_DN` - ISI of the DN train in the Fast Ripple band during normal activity

## Let's analyze the Ripple Band first

In [9]:
isi_ripple_hfo_up = []
isi_ripple_hfo_down = []
isi_ripple_noise_up = []
isi_ripple_noise_down = []

#### Ripple Band - UP Train

In [10]:
from utils.input import RIPPLE_CONFIDENCE_WINDOW, FR_CONFIDENCE_WINDOW, BOTH_CONFIDENCE_WINDOW

# Iterate the Ripple UP Spike Train
curr_ripple_gt_idx = 0  # Current index of the ripple ground truth
prev_spike_time = ripple_up_spike_train[0][0]

for spike_idx in range(1, len(ripple_up_spike_train), 1):
    spike_time = ripple_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_ripple_gt_idx < len(ripple_ground_truth) and spike_time > ripple_ground_truth[curr_ripple_gt_idx][1] + RIPPLE_CONFIDENCE_WINDOW:
        curr_ripple_gt_idx += 1
    
    # Check if the current spike time is within the ripple ground truth
    if curr_ripple_gt_idx < len(ripple_ground_truth):
        curr_gt_start = ripple_ground_truth[curr_ripple_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + RIPPLE_CONFIDENCE_WINDOW:
            # The spike is within the ripple ground truth
            isi_ripple_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the ripple ground truth
    isi_ripple_noise_up.append(isi)

#### Ripple Band - DOWN Train

In [11]:
# Iterate the Ripple DOWN Spike Train
curr_ripple_gt_idx = 0  # Current index of the ripple ground truth
prev_spike_time = ripple_down_spike_train[0][0]

for spike_idx in range(1, len(ripple_down_spike_train), 1):
    spike_time = ripple_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_ripple_gt_idx < len(ripple_ground_truth) and spike_time > ripple_ground_truth[curr_ripple_gt_idx][1] + RIPPLE_CONFIDENCE_WINDOW:
        curr_ripple_gt_idx += 1
    
    # Check if the current spike time is within the ripple ground truth
    if curr_ripple_gt_idx < len(ripple_ground_truth):
        curr_gt_start = ripple_ground_truth[curr_ripple_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + RIPPLE_CONFIDENCE_WINDOW:
            # The spike is within the ripple ground truth
            isi_ripple_hfo_down.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the ripple ground truth
    isi_ripple_noise_down.append(isi)

### Validate the number of calculated ISIs
Validate that:
- `len(isi_ripple_hfo_up)` + `len(isi_ripple_noise_up)` = `len(ripple_up)` - 1 (first spike does not have an ISI)
- `len(isi_ripple_hfo_down_)` + `len(isi_ripple_noise_down_)` = `len(ripple_down_)` - 1 (first spike does not have an ISI)

In [12]:
#  Validate that len(isi_ripple_hfo_up) + len(isi_ripple_noise_up) = len(ripple_up)
print(f"Length isi_ripple_hfo_up: {len(isi_ripple_hfo_up)}")
print(f"Length isi_ripple_noise_up: {len(isi_ripple_noise_up)}")
print(f"Sum: {len(isi_ripple_hfo_up) + len(isi_ripple_noise_up)}")
print(f"Length ripple_up: {len(ripple_up_spike_train)}")

Length isi_ripple_hfo_up: 2075
Length isi_ripple_noise_up: 469
Sum: 2544
Length ripple_up: 2545


In [13]:
#  Validate that len(isi_ripple_hfo_down) + len(isi_ripple_noise_down) = len(ripple_down)
print(f"Length isi_ripple_hfo_down: {len(isi_ripple_hfo_down)}")
print(f"Length isi_ripple_noise_down: {len(isi_ripple_noise_down)}")
print(f"Sum: {len(isi_ripple_hfo_down) + len(isi_ripple_noise_down)}")
print(f"Length ripple_down: {len(ripple_down_spike_train)}")

Length isi_ripple_hfo_down: 2044
Length isi_ripple_noise_down: 485
Sum: 2529
Length ripple_down: 2530


## Let's analyze the Fast Ripple Band next

In [14]:
isi_fr_hfo_up = []
isi_fr_hfo_down = []
isi_fr_noise_up = []
isi_fr_noise_down = []

#### Fast Ripple Band - UP Train

In [15]:
# Iterate the FR UP Spike Train
curr_fr_gt_idx = 0  # Current index of the fast ripple ground truth
prev_spike_time = fr_up_spike_train[0][0]

for spike_idx in range(1, len(fr_up_spike_train), 1):
    spike_time = fr_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the fast ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_fr_gt_idx < len(fr_ground_truth) and spike_time > fr_ground_truth[curr_fr_gt_idx][1] + FR_CONFIDENCE_WINDOW:
        curr_fr_gt_idx += 1
    
    # Check if the current spike time is within the fast ripple ground truth
    if curr_fr_gt_idx < len(fr_ground_truth):
        curr_gt_start = fr_ground_truth[curr_fr_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + FR_CONFIDENCE_WINDOW:
            # The spike is within the fast ripple ground truth
            isi_fr_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the fast ripple ground truth
    isi_fr_noise_up.append(isi)

#### Fast Ripple Band - DN Train

In [16]:
# Iterate the FR UP Spike Train
curr_fr_gt_idx = 0  # Current index of the fast ripple ground truth
prev_spike_time = fr_down_spike_train[0][0]

for spike_idx in range(1, len(fr_down_spike_train), 1):
    spike_time = fr_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the fast ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_fr_gt_idx < len(fr_ground_truth) and spike_time > fr_ground_truth[curr_fr_gt_idx][1] + FR_CONFIDENCE_WINDOW:
        curr_fr_gt_idx += 1
    
    # Check if the current spike time is within the fast ripple ground truth
    if curr_fr_gt_idx < len(fr_ground_truth):
        curr_gt_start = fr_ground_truth[curr_fr_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + FR_CONFIDENCE_WINDOW:
            # The spike is within the fast ripple ground truth
            isi_fr_hfo_down.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the fast ripple ground truth
    isi_fr_noise_down.append(isi)

### Validate the number of calculated ISIs
Validate that:
- `len(isi_fr_hfo_up)` + `len(isi_fr_noise_up)` = `len(fr_up)` - 1 (first spike does not have an ISI)
- `len(isi_fr_hfo_down)` + `len(isi_fr_noise_down)` = `len(fr_down)` - 1 (first spike does not have an ISI)

In [17]:
#  Validate that len(isi_fr_hfo_up) + len(isi_fr_noise_up) = len(ripple_up)
print(f"Length isi_fr_hfo_up: {len(isi_fr_hfo_up)}")
print(f"Length isi_fr_noise_up: {len(isi_fr_noise_up)}")
print(f"Sum: {len(isi_fr_hfo_up) + len(isi_fr_noise_up)}")
print(f"Length fr_up: {len(fr_up_spike_train)}")

Length isi_fr_hfo_up: 2336
Length isi_fr_noise_up: 1634
Sum: 3970
Length fr_up: 3971


In [18]:
#  Validate that len(isi_fr_hfo_down) + len(isi_fr_noise_down) = len(ripple_down)
print(f"Length isi_fr_hfo_down: {len(isi_fr_hfo_down)}")
print(f"Length isi_fr_noise_down: {len(isi_fr_noise_down)}")
print(f"Sum: {len(isi_fr_hfo_down) + len(isi_fr_noise_down)}")
print(f"Length fr_down: {len(fr_down_spike_train)}")

Length isi_fr_hfo_down: 2474
Length isi_fr_noise_down: 1570
Sum: 4044
Length fr_down: 4045


--- 

## Calculate metrics of the ISIs and Show the Results

In [19]:
# Ripple band ISIs
# UP
# ---- Relevant Event Metrics ----
mean_ripple_hfo_up = np.mean(isi_ripple_hfo_up)
median_ripple_hfo_up = np.median(isi_ripple_hfo_up)
std_ripple_hfo_up = np.std(isi_ripple_hfo_up)
# Calculate the IQR for the ISI in the Ripple Band for the UP spikes during HFO
q1_ripple_hfo_up = np.percentile(isi_ripple_hfo_up, 25)
q3_ripple_hfo_up = np.percentile(isi_ripple_hfo_up, 75)
iqr_ripple_hfo_up = q3_ripple_hfo_up - q1_ripple_hfo_up

# ---- Baseline Activity Metrics ----
mean_ripple_noise_up = np.mean(isi_ripple_noise_up)
median_ripple_noise_up = np.median(isi_ripple_noise_up)
std_ripple_noise_up = np.std(isi_ripple_noise_up)
# Calculate the IQR for the ISI in the Ripple Band for the UP spikes during Noise
q1_ripple_noise_up = np.percentile(isi_ripple_noise_up, 25)
q3_ripple_noise_up = np.percentile(isi_ripple_noise_up, 75)
iqr_ripple_noise_up = q3_ripple_noise_up - q1_ripple_noise_up

# DOWN
# ---- Relevant Event Metrics ----
mean_ripple_hfo_down = np.mean(isi_ripple_hfo_down)
median_ripple_hfo_down = np.median(isi_ripple_hfo_down)
std_ripple_hfo_down = np.std(isi_ripple_hfo_down)
# Calculate the IQR for the ISI in the Ripple Band for the DOWN spikes during HFO
q1_ripple_hfo_down = np.percentile(isi_ripple_hfo_down, 25)
q3_ripple_hfo_down = np.percentile(isi_ripple_hfo_down, 75)
iqr_ripple_hfo_down = q3_ripple_hfo_down - q1_ripple_hfo_down

# ---- Baseline Activity Metrics ----
mean_ripple_noise_down = np.mean(isi_ripple_noise_down)
median_ripple_noise_down = np.median(isi_ripple_noise_down)
std_ripple_noise_down = np.std(isi_ripple_noise_down)
# Calculate the IQR for the ISI in the Ripple Band for the DOWN spikes during Noise
q1_ripple_noise_down = np.percentile(isi_ripple_noise_down, 25)
q3_ripple_noise_down = np.percentile(isi_ripple_noise_down, 75)
iqr_ripple_noise_down = q3_ripple_noise_down - q1_ripple_noise_down

In [20]:
# Fast Ripple band ISIs
# UP
# ---- Relevant Event Metrics ----
mean_fr_hfo_up = np.mean(isi_fr_hfo_up)
median_fr_hfo_up = np.median(isi_fr_hfo_up)
std_fr_hfo_up = np.std(isi_fr_hfo_up)
# Calculate the IQR for the ISI in the Fast Ripple Band for the UP spikes during HFO
q1_fr_hfo_up = np.percentile(isi_fr_hfo_up, 25)
q3_fr_hfo_up = np.percentile(isi_fr_hfo_up, 75)
iqr_fr_hfo_up = q3_fr_hfo_up - q1_fr_hfo_up

# ---- Baseline Activity Metrics ----
mean_fr_noise_up = np.mean(isi_fr_noise_up)
median_fr_noise_up = np.median(isi_fr_noise_up)
std_fr_noise_up = np.std(isi_fr_noise_up)
# Calculate the IQR for the ISI in the Fast Ripple Band for the UP spikes during Noise
q1_fr_noise_up = np.percentile(isi_fr_noise_up, 25)
q3_fr_noise_up = np.percentile(isi_fr_noise_up, 75)
iqr_fr_noise_up = q3_fr_noise_up - q1_fr_noise_up

# DOWN
# ---- Relevant Event Metrics ----
mean_fr_hfo_down = np.mean(isi_fr_hfo_down)
median_fr_hfo_down = np.median(isi_fr_hfo_down)
std_fr_hfo_down = np.std(isi_fr_hfo_down)
# Calculate the IQR for the ISI in the Fast Ripple Band for the DOWN spikes during HFO
q1_fr_hfo_down = np.percentile(isi_fr_hfo_down, 25)
q3_fr_hfo_down = np.percentile(isi_fr_hfo_down, 75)
iqr_fr_hfo_down = q3_fr_hfo_down - q1_fr_hfo_down

# ---- Baseline Activity Metrics ----
mean_fr_noise_down = np.mean(isi_fr_noise_down)
median_fr_noise_down = np.median(isi_fr_noise_down)
std_fr_noise_down = np.std(isi_fr_noise_down)
# Calculate the IQR for the ISI in the Fast Ripple Band for the DOWN spikes during Noise
q1_fr_noise_down = np.percentile(isi_fr_noise_down, 25)
q3_fr_noise_down = np.percentile(isi_fr_noise_down, 75)
iqr_fr_noise_down = q3_fr_noise_down - q1_fr_noise_down

### Print the Metrics of the ISIs

#### Ripple Band

In [21]:
# Metrics of the ISIs in the Ripple Band
print("Ripple UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_ripple_hfo_up}, Median: {median_ripple_hfo_up}, STD: {std_ripple_hfo_up}, IQR: [{q1_ripple_hfo_up} - {q3_ripple_hfo_up}]")
print("Noise ISI (ms)")
print(f"Mean: {mean_ripple_noise_up}, Median: {median_ripple_noise_up}, STD: {std_ripple_noise_up}, IQR: [{q1_ripple_noise_up} - {q3_ripple_noise_up}]")

print("=================================")
print("Ripple DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_ripple_hfo_down}, Median: {median_ripple_hfo_down}, STD: {std_ripple_hfo_down}, IQR: [{q1_ripple_hfo_down} - {q3_ripple_hfo_down}]")
print("Noise ISI (ms)")
print(f"Mean: {mean_ripple_noise_down}, Median: {median_ripple_noise_down}, STD: {std_ripple_noise_down}, IQR: [{q1_ripple_noise_down}  - {q3_ripple_noise_down}]")

Ripple UP
HFO ISI (ms)
Mean: 26.57403049698795, Median: 1.46484375, STD: 114.40573201043128, IQR: [0.9765625 - 8.7890625]
Noise ISI (ms)
Mean: 134.37312599946696, Median: 10.7421875, STD: 227.86356398108373, IQR: [0.9765625 - 229.4921875]
Ripple DOWN
HFO ISI (ms)
Mean: 26.362887567270057, Median: 1.46484375, STD: 116.264527680139, IQR: [0.9765625 - 8.7890625]
Noise ISI (ms)
Mean: 132.5094635953608, Median: 10.7421875, STD: 218.102061220564, IQR: [0.9765625  - 238.28125]


#### Fast Ripple Band

In [22]:
# Metrics of the ISIs in the FR Band
print("FR UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_fr_hfo_up}, Median: {median_fr_hfo_up}, STD: {std_fr_hfo_up}, IQR: [{q1_fr_hfo_up} - {q3_fr_hfo_up}]")
print("Noise ISI (ms)")
print(f"Mean: {mean_fr_noise_up}, Median: {median_fr_noise_up}, STD: {std_fr_noise_up}, IQR: [{q1_fr_noise_up} - {q3_fr_noise_up}]")

print("=================================")
print("FR DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_fr_hfo_down}, Median: {median_fr_hfo_down}, STD: {std_fr_hfo_down}, IQR: [{q1_fr_hfo_down} - {q3_fr_hfo_down}]")
print("Noise ISI (ms)")
print(f"Mean: {mean_fr_noise_down}, Median: {median_fr_noise_down}, STD: {std_fr_noise_down}, IQR: [{q1_fr_noise_down} - {q3_fr_noise_down}]")

FR UP
HFO ISI (ms)
Mean: 2.677603943707192, Median: 2.44140625, STD: 20.13498106566833, IQR: [0.48828125 - 2.9296875]
Noise ISI (ms)
Mean: 68.39583173959608, Median: 2.44140625, STD: 170.88806575131488, IQR: [1.953125 - 20.5078125]
FR DOWN
HFO ISI (ms)
Mean: 3.0978425626515764, Median: 2.44140625, STD: 24.13413060017164, IQR: [0.48828125 - 2.9296875]
Noise ISI (ms)
Mean: 70.28761942675159, Median: 2.44140625, STD: 173.6208426511047, IQR: [1.953125 - 21.8505859375]


### Display Box Plots with the Inter-Spike Intervals

#### Ripple ISIs

In [23]:
# Display Box Plots with the Inter-Spike Intervals
from utils.bar_plot import create_box_plot
import bokeh.plotting as bplt

# Ripple Band Box Plot
# Define the array of values
ripple_isi_list = [isi_ripple_hfo_up, isi_ripple_hfo_down, isi_ripple_noise_up, isi_ripple_noise_down]
# Create the BoxPlot
ripple_isi_boxplot = create_box_plot(
    title="Inter-Spike Interval of the UP/DOWN Spike trains in the Ripple Band", 
    box_arrays=ripple_isi_list,
    y_axis_label='Inter-Spike Interval (ms)',
    x_axis_labels=['HFO UP', 'HFO DOWN', 'NOISE UP', 'NOISE DOWN'],
    sizing_mode="stretch_width",
)
# TODO: Add legend on the x-axis

show_ripple_isi_barplot = True
if show_ripple_isi_barplot:
    # Show the plot
    bplt.show(ripple_isi_boxplot)

ripple_quantiles:  [0.9765625  1.46484375 8.7890625 ]
Ripple IQR:  7.8125
ripple_quantiles:  [0.9765625  1.46484375 8.7890625 ]
Ripple IQR:  7.8125
ripple_quantiles:  [  0.9765625  10.7421875 229.4921875]
Ripple IQR:  228.515625
ripple_quantiles:  [  0.9765625  10.7421875 238.28125  ]
Ripple IQR:  237.3046875


In [24]:
# Export the plot to an HTML file
EXPORT_RIPPLE_BOXPLOT = True
if EXPORT_RIPPLE_BOXPLOT:
    # Create the folder if it does not exist
    if not os.path.exists(RESULTS_FOLDER):
        os.makedirs(RESULTS_FOLDER)

    ripple_boxplot_filename = f"{RESULTS_FOLDER}/ripple_isi_boxplot_thresh{ripple_thresh_up}-{ripple_thresh_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=ripple_boxplot_filename, title="Inter-Spike Interval of the UP/DOWN Spike trains in the Ripple Band")

    # Save the plot
    bplt.save(ripple_isi_boxplot)

    # Close the plot
    bplt.reset_output()

#### Fast Ripple ISIs

In [25]:
# Display Box Plots with the Inter-Spike Intervals

# FR Band Box Plot
# Define the array of values
fr_isi_list = [isi_fr_hfo_up, isi_fr_hfo_down, isi_fr_noise_up, isi_fr_noise_down]
# Create the BoxPlot
fr_isi_boxplot = create_box_plot(
    title="Inter-Spike Interval of the UP/DOWN Spike trains in the Fast Ripple Band", 
    box_arrays=fr_isi_list,
    y_axis_label='Inter-Spike Interval (ms)',
    x_axis_labels=['HFO UP', 'HFO DOWN', 'NOISE UP', 'NOISE DOWN'],
    sizing_mode="stretch_width",
)
# TODO: Add legend on the x-axis

show_fr_isi_barplot = True
if show_fr_isi_barplot:
    # Show the plot
    bplt.show(fr_isi_boxplot)

ripple_quantiles:  [0.48828125 2.44140625 2.9296875 ]
Ripple IQR:  2.44140625
ripple_quantiles:  [0.48828125 2.44140625 2.9296875 ]
Ripple IQR:  2.44140625
ripple_quantiles:  [ 1.953125    2.44140625 20.5078125 ]
Ripple IQR:  18.5546875
ripple_quantiles:  [ 1.953125    2.44140625 21.85058594]
Ripple IQR:  19.8974609375


In [26]:
# Export the plot to an HTML file
EXPORT_FR_PLOT = True
if EXPORT_FR_PLOT:
    # Create the folder if it does not exist
    if not os.path.exists(RESULTS_FOLDER):
        os.makedirs(RESULTS_FOLDER)

    fr_boxplot_filename = f"{RESULTS_FOLDER}/fr_isi_boxplot_thresh{fr_thresh_up}-{fr_thresh_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=fr_boxplot_filename, title="Inter-Spike Interval of the UP/DOWN Spike trains in the Fast Ripple Band")

    # Save the plot
    bplt.save(fr_isi_boxplot)

    bplt.reset_output()